Autor: Kaike Castro Carvalho

Date: 26-May-2024

# Import Libraries

In [1]:
import requests
import urllib3
from requests_kerberos import HTTPKerberosAuth
import json

## Base Code

In [ ]:
class VisionApi:
    def __init__(self, url_endpoint, username, password):
        self.url_endpoint = url_endpoint
        self.username = username
        self.password = password
        self.session = requests.Session()
        self.session.verify = False  # Be cautious with this in a production environment

    def _request(self, method, custom_url, body=None):
        """
        Generic request method to handle all types of HTTP requests.
        """
        # Set up authentication and headers
        auth = HttpNtlmAuth(self.username, self.password)
        headers = {
            'Content-Type': 'application/json',
            'X-Requested-With': 'XMLHttpRequest'
        }

        # Construct the full API URL
        api_url = f"{self.url_endpoint}/{custom_url}"

        # Try to make the HTTP request
        try:
            if method.upper() == 'GET':
                response = self.session.get(api_url, auth=auth, headers=headers)
            elif method.upper() == 'POST':
                response = self.session.post(api_url, auth=auth, headers=headers, json=body)
            elif method.upper() == 'PUT':
                response = self.session.put(api_url, auth=auth, headers=headers, json=body)
            elif method.upper() == 'DELETE':
                response = self.session.delete(api_url, auth=auth, headers=headers)
            else:
                raise ValueError("Unsupported HTTP method provided.")

            # Handle responses
            if response.status_code == 200:
                return response.json()
            if response.status_code == 204:
                print(f"204 No Content: No content returned from the API: {api_url}")
                return None
            print(f"{response.status_code} Error: {response.text} from the API: {api_url}")
            return None

        except requests.exceptions.RequestException as e:
            print(f"Connection Error while accessing the API: {api_url}: {e}")
            return None

    def get_pi_vision_api(self, custom_url):
        return self._request("GET", custom_url)

    def post_pi_vision_api(self, custom_url, data):
        return self._request("POST", custom_url, body=data)

    def put_pi_vision_api(self, custom_url, data):
        return self._request("PUT", custom_url, body=data)

    def delete_pi_vision_api(self, custom_url):
        return self._request("DELETE", custom_url)


In [ ]:
api = VisionApi("https://<servername>/PIVision/Utility/api/v1", "<username>", "<password>")

# Applying Templates (in development)

In [ ]:
def apply_template(display_id, template_id):
       
    template = api.get_PiVisionApi(f"displays/{template_id}/export")        
    display = api.get_PiVisionApi(f"displays/{display_id}/export")

    display_copy = display.copy() 
    
    top_adjustment = 200
    
    # 1. Adjust 'Top' 
    for symbol in display_copy["Display"]["Symbols"]:
        if "Configuration" in symbol:
            top = float(symbol["Configuration"].get("Top", 0))
            symbol["Configuration"]["Top"] = top + top_adjustment

    max_top_symbol = max((symbol for symbol in data["Display"]["Symbols"] if "Configuration" in symbol and "Top" in symbol["Configuration"]),
                        key=lambda s: float(s["Configuration"]["Top"]))
    max_top = float(max_top_symbol["Configuration"]["Top"])
    max_height = float(max_top_symbol["Configuration"]["Height"]) 

    # 2. Get images from template        

    image_symbol = next((symbol for symbol in template["Display"]["Symbols"] if symbol["SymbolType"] == "image"), None)
    
    # 3. Get the last symbol number and name
    
    last_symbol = data["Display"]["Symbols"][-1]
    last_number = int(last_symbol["Name"].replace('Symbol', '')) + 1

    # 4. Rename the image symbol and add it to the display
    image_symbol["Name"] = "Symbol" + str(last_number)
    
    # 5. Add the image symbol to the display
    display_copy["Display"]["Symbols"].append(image_symbol)
    
    display_copy['Display']['AttachmentIds'] = template['Display']['AttachmentIds']
    display_copy["Attachments"] = template.get("Attachments", [])
                    

    for symbol in template["Display"]["Symbols"]:
        last_number += 1
        if symbol["SymbolType"] == "statictext":         
            symbol["Name"] = "Symbol" + str(last_number)        
        elif symbol["SymbolType"] == "line":
            if symbol["Name"] == "bottom":
                symbol["Configuration"]["Top"] = str(max_top + max_height + 50.0)
            symbol["Name"] = "Symbol" + str(last_number)
        display_copy["Display"]["Symbols"].append(symbol.copy())
   
    